In [8]:
import cv2
import numpy as np
import os
import re
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline 

In [9]:
dataSet = []
person = []
horiSet = []
train_set_x = []
count = 0
folder = "/home/jeet/CV Assignment/ML face images train"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
#         print(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200,200))
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(img,None)
        kmeans = KMeans(n_clusters=1, random_state=0).fit(des)
#         print(kmeans.cluster_centers_.shape)
        train_set_x.append(kmeans.cluster_centers_)
#         person.append(numbers[0])
        img = np.ravel(img)
        if img is not None:
#             for desc in des: 
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person.append(numbers[0])
            horiSet.append(img.T)
#             dataSet.append(desc)
            count = count + 1

print(np.array(train_set_x).shape)      
dataSet_test = []
person_test = []
horiSet_test = []
test_set_x = []
folder = "/home/jeet/CV Assignment/ML face images test"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200, 200))
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(img,None)
        #kp will be a list of keypoints and des is a numpy array of shape Number_of_Keypoints×128.
        kmeans = KMeans(n_clusters=1, random_state=0).fit(des)
        test_set_x.append(kmeans.cluster_centers_)
        img = np.ravel(img)
        if img is not None:
#             for desc in des: 
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person_test.append(numbers[0])
            horiSet_test.append(img.T)
#             dataSet_test.append(desc)
            count = count + 1
print(np.array(test_set_x).shape , np.array(person_test).shape)

(204, 1, 128)
(70, 1, 128) (70,)


In [10]:
train_set_x = np.reshape(train_set_x,(418,128))
test_set_x = np.reshape(test_set_x,(140,128))

In [11]:
import pandas as pd

#Getting data in desired format and mapping the lables
dataSet = np.array(train_set_x)
train = pd.DataFrame(dataSet)

person = np.array(person)
train['id'] = person.transpose()

unique_person = train['id']
unique_person = np.array(unique_person)
unique_person = np.unique(unique_person)
key = range(len(unique_person))

roll_dict = dict( zip(unique_person, key))

mapped_roll = np.vectorize(roll_dict.get)(person)
train['id'] = mapped_roll.transpose()

y = train['id'].values.astype('int64')
images = train.drop(['id'], axis=1, inplace=False)
x = (images.values).astype('uint8')

X_train = x
Y_train = y

dataSet_test = np.array(test_set_x)
test = pd.DataFrame(dataSet_test)

person_test = np.array(person_test)
test['id'] = person_test.transpose()

mapped_roll_test = np.vectorize(roll_dict.get)(person_test)
test['id'] = mapped_roll_test.transpose()
print(test['id'])

# temp = np_utils.to_categorical(test['id'])
# print(temp[2])
# print(temp.shape)


y_test = test['id'].values.astype('int64')
images = test.drop(['id'], axis=1, inplace=False)
x_test = (images.values).astype('uint8')

X_test = x_test
Y_test = y_test



0     28
1     31
2      4
3     13
4     33
5     34
6     28
7     24
8     18
9     32
10     2
11     6
12    30
13    19
14     4
15    21
16    29
17    14
18    15
19    20
20    14
21    10
22    15
23    30
24     7
25    18
26    23
27    17
28    24
29    12
      ..
40     2
41    21
42     3
43     6
44     9
45    11
46    11
47    26
48     7
49    27
50    27
51     0
52    33
53    34
54    22
55    16
56     8
57    32
58    25
59    22
60    26
61    13
62     9
63     0
64     5
65    16
66    17
67     5
68    29
69     8
Name: id, Length: 70, dtype: int64


In [12]:
print(X_test.shape)

(70, 128)


In [13]:
print(X_train.shape, Y_train.shape)

(204, 128) (204,)


In [14]:
from sklearn import svm


print("Training Started...")
clf = svm.SVC(kernel="linear", C=100.0, gamma ="auto") #initialising the svm #classifier
#clf = svm.SVC(gamma=0.001, C = 100)
clf.fit(X_train,Y_train) #fitting and training

print( "Training Finished...")

Training Started...
Training Finished...


In [15]:
score = clf.score(X_test, Y_test)

In [16]:
score

0.8285714285714286